In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

ImportError: cannot import name 'tensor' from 'tensorflow.python.framework' (/Users/shikaiyi/opt/anaconda3/envs/StockMarketIndexAI/lib/python3.11/site-packages/tensorflow/python/framework/__init__.py)

In [2]:
df = pd.read_csv('../data/row_data.csv')

In [3]:
df.columns

Index(['date', '台積_SMA10', '台積_SMA5', '台積_收盤價', '台積_最低價', '台積_最高價', '台積_開盤價',
       '外資', '大盤_SMA10', '大盤_SMA5', '大盤_成交量', '大盤_收盤價', '大盤_最低價', '大盤_最高價',
       '大盤_開盤價', '投信', '日圓_SMA10', '日圓_SMA5', '日圓_收盤價', '日圓_最低價', '日圓_最高價',
       '日圓_開盤價', '美元_SMA10', '美元_SMA5', '美元_收盤價', '美元_最低價', '美元_最高價', '美元_開盤價',
       '自營商', '費半_SMA10', '費半_SMA5', '費半_收盤價', '費半_最低價', '費半_最高價', '費半_開盤價'],
      dtype='object')

In [ ]:
mple data
# df = pd.read_csv('your_data.csv')

# Data Preprocessing
# Convert 'date' column to datetime type
df['date'] = pd.to_datetime(df['date'])

# Sort data by date
df = df.sort_values(by='date')

# Drop 'date' column for now
df = df.drop(['date'], axis=1)

# Ensure no missing values
df = df.dropna()

# Normalize data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)

# Create sequences for LSTM
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        seq = data[i:i + seq_length]
        X.append(seq[:-1])
        y.append(seq[-1][df.columns.get_loc("大盤_收盤價")])
    return np.array(X), np.array(y)

seq_length = 10  # for example, use the last 10 days to predict the next day
X, y = create_sequences(scaled_data, seq_length)

# Split data into training and testing (80-20 split for example)
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Model Creation
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Model Training
model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))

# Predictions
predictions = model.predict(X_test)
